In [1]:
import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr
from xgboost import XGBRegressor

from sklearn.linear_model import LinearRegression
from tqdm import tqdm

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
path = os.getcwd()
list_name = 'stock_list.csv'
sample_name = 'sample_submission.csv'

stock_list = pd.read_csv(os.path.join(path, list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [3]:
start_date = '20210104'
end_date = '20211214'

#
Business_days = pd.DataFrame(pd.date_range(start_date, end_date, freq='B'), columns = ['Date'])

In [4]:
Business_days

,Date
0,2021-01-04
1,2021-01-05
2,2021-01-06
3,2021-01-07
4,2021-01-08
...,...
242,2021-12-08
243,2021-12-09
244,2021-12-10
245,2021-12-13


In [5]:
#요일,주차 표기
Business_days['weekday'] = Business_days.Date.apply(lambda x:x.weekday())
Business_days['weeknum'] = Business_days.Date.apply(lambda x:x.strftime('%V'))
Business_days

,Date,weekday,weeknum
0,2021-01-04,0,01
1,2021-01-05,1,01
2,2021-01-06,2,01
3,2021-01-07,3,01
4,2021-01-08,4,01
...,...,...,...
242,2021-12-08,2,49
243,2021-12-09,3,49
244,2021-12-10,4,49
245,2021-12-13,0,50


In [6]:
# column rename하는 함수
def col_rename(data_set):
    for i in data_set.columns:
        if i =='Date':
            pass
        else:
            data_set.rename(columns={i:data_set.name+'_'+i}, inplace=True)
    return data_set  

In [7]:
#KOSPI 정보 추가
KOSPI = fdr.DataReader('KS11', start_date, end_date).reset_index()
KOSPI.name = 'kospi'

In [8]:
col_rename(KOSPI)

,Date,kospi_Close,kospi_Open,kospi_High,kospi_Low,kospi_Volume,kospi_Change
0,2021-01-04,2944.45,2874.50,2946.54,2869.11,1.030000e+09,0.0247
1,2021-01-05,2990.57,2943.67,2990.57,2921.84,1.520000e+09,0.0157
2,2021-01-06,2968.21,2993.34,3027.16,2961.37,1.790000e+09,-0.0075
3,2021-01-07,3031.68,2980.75,3055.28,2980.75,1.520000e+09,0.0214
4,2021-01-08,3152.18,3040.11,3161.11,3040.11,1.300000e+09,0.0397
...,...,...,...,...,...,...,...
231,2021-12-08,3001.80,3017.93,3036.13,2995.34,4.882500e+08,0.0034
232,2021-12-09,3029.57,3007.00,3029.57,3001.55,5.134400e+08,0.0093
233,2021-12-10,3010.23,3008.70,3017.64,2998.29,4.516000e+08,-0.0064
234,2021-12-13,3001.66,3019.67,3043.83,3000.51,3.758300e+08,-0.0028


In [9]:
# 데이터 묶기
data = pd.merge(Business_days, KOSPI, how = 'outer')
data

,Date,weekday,weeknum,kospi_Close,kospi_Open,kospi_High,kospi_Low,kospi_Volume,kospi_Change
0,2021-01-04,0,01,2944.45,2874.50,2946.54,2869.11,1.030000e+09,0.0247
1,2021-01-05,1,01,2990.57,2943.67,2990.57,2921.84,1.520000e+09,0.0157
2,2021-01-06,2,01,2968.21,2993.34,3027.16,2961.37,1.790000e+09,-0.0075
3,2021-01-07,3,01,3031.68,2980.75,3055.28,2980.75,1.520000e+09,0.0214
4,2021-01-08,4,01,3152.18,3040.11,3161.11,3040.11,1.300000e+09,0.0397
...,...,...,...,...,...,...,...,...,...
242,2021-12-08,2,49,3001.80,3017.93,3036.13,2995.34,4.882500e+08,0.0034
243,2021-12-09,3,49,3029.57,3007.00,3029.57,3001.55,5.134400e+08,0.0093
244,2021-12-10,4,49,3010.23,3008.70,3017.64,2998.29,4.516000e+08,-0.0064
245,2021-12-13,0,50,3001.66,3019.67,3043.83,3000.51,3.758300e+08,-0.0028


In [10]:
#KODSAQ 정보 추가
KOSDAQ = fdr.DataReader('KQ11', start_date, end_date).reset_index()
KOSDAQ.name = 'kosdaq'
col_rename(KOSDAQ)

,Date,kosdaq_Close,kosdaq_Open,kosdaq_High,kosdaq_Low,kosdaq_Volume,kosdaq_Change
0,2021-01-04,977.62,968.86,977.62,960.52,1.700000e+09,0.0095
1,2021-01-05,985.76,976.43,985.76,965.53,1.810000e+09,0.0083
2,2021-01-06,981.39,987.25,990.88,977.37,1.980000e+09,-0.0044
3,2021-01-07,988.86,983.28,993.91,982.27,2.260000e+09,0.0076
4,2021-01-08,987.79,990.70,995.22,978.12,2.560000e+09,-0.0011
...,...,...,...,...,...,...,...
231,2021-12-08,1006.04,1006.61,1012.64,1003.92,1.070000e+09,0.0094
232,2021-12-09,1022.87,1009.29,1022.87,1009.29,1.060000e+09,0.0167
233,2021-12-10,1011.57,1016.34,1018.42,1010.02,1.150000e+09,-0.0110
234,2021-12-13,1005.96,1014.27,1014.90,1005.96,1.340000e+09,-0.0055


In [11]:
data = pd.merge(data, KOSDAQ, how = 'outer')
data

,Date,weekday,weeknum,kospi_Close,kospi_Open,kospi_High,kospi_Low,kospi_Volume,kospi_Change,kosdaq_Close,kosdaq_Open,kosdaq_High,kosdaq_Low,kosdaq_Volume,kosdaq_Change
0,2021-01-04,0,01,2944.45,2874.50,2946.54,2869.11,1.030000e+09,0.0247,977.62,968.86,977.62,960.52,1.700000e+09,0.0095
1,2021-01-05,1,01,2990.57,2943.67,2990.57,2921.84,1.520000e+09,0.0157,985.76,976.43,985.76,965.53,1.810000e+09,0.0083
2,2021-01-06,2,01,2968.21,2993.34,3027.16,2961.37,1.790000e+09,-0.0075,981.39,987.25,990.88,977.37,1.980000e+09,-0.0044
3,2021-01-07,3,01,3031.68,2980.75,3055.28,2980.75,1.520000e+09,0.0214,988.86,983.28,993.91,982.27,2.260000e+09,0.0076
4,2021-01-08,4,01,3152.18,3040.11,3161.11,3040.11,1.300000e+09,0.0397,987.79,990.70,995.22,978.12,2.560000e+09,-0.0011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,2021-12-08,2,49,3001.80,3017.93,3036.13,2995.34,4.882500e+08,0.0034,1006.04,1006.61,1012.64,1003.92,1.070000e+09,0.0094
243,2021-12-09,3,49,3029.57,3007.00,3029.57,3001.55,5.134400e+08,0.0093,1022.87,1009.29,1022.87,1009.29,1.060000e+09,0.0167
244,2021-12-10,4,49,3010.23,3008.70,3017.64,2998.29,4.516000e+08,-0.0064,1011.57,1016.34,1018.42,1010.02,1.150000e+09,-0.0110
245,2021-12-13,0,50,3001.66,3019.67,3043.83,3000.51,3.758300e+08,-0.0028,1005.96,1014.27,1014.90,1005.96,1.340000e+09,-0.0055


In [12]:
#다우지수 추가 : 4/10 20:00 뺴기로 결정
#DJI = fdr.DataReader('dji', start_date, end_date).reset_index()
#DJI.name = 'dji'
#col_rename(DJI)

In [13]:
#data = pd.merge(data, DJI, how = 'outer')
#data

In [14]:
#원/달러 환율 추가
EXC = fdr.DataReader('USD/KRW', start_date, end_date).reset_index()
EXC.name = 'exc'
col_rename(EXC)

,Date,exc_Close,exc_Open,exc_High,exc_Low,exc_Change
0,2021-01-04,1086.48,1085.73,1088.30,1080.02,0.0016
1,2021-01-05,1086.42,1086.69,1090.33,1082.04,-0.0001
2,2021-01-06,1087.93,1087.40,1089.79,1083.91,0.0014
3,2021-01-07,1094.28,1088.03,1096.78,1085.42,0.0058
4,2021-01-08,1092.93,1094.35,1099.21,1088.79,-0.0012
...,...,...,...,...,...,...
242,2021-12-08,1175.19,1176.85,1179.31,1175.12,-0.0013
243,2021-12-09,1178.15,1173.42,1179.37,1172.62,0.0025
244,2021-12-10,1180.86,1178.27,1182.82,1176.40,0.0023
245,2021-12-13,1184.91,1180.96,1186.16,1177.03,0.0034


In [15]:
data = pd.merge(data, EXC, how = 'outer')
data

,Date,weekday,weeknum,kospi_Close,kospi_Open,kospi_High,kospi_Low,kospi_Volume,kospi_Change,kosdaq_Close,kosdaq_Open,kosdaq_High,kosdaq_Low,kosdaq_Volume,kosdaq_Change,exc_Close,exc_Open,exc_High,exc_Low,exc_Change
0,2021-01-04,0,01,2944.45,2874.50,2946.54,2869.11,1.030000e+09,0.0247,977.62,968.86,977.62,960.52,1.700000e+09,0.0095,1086.48,1085.73,1088.30,1080.02,0.0016
1,2021-01-05,1,01,2990.57,2943.67,2990.57,2921.84,1.520000e+09,0.0157,985.76,976.43,985.76,965.53,1.810000e+09,0.0083,1086.42,1086.69,1090.33,1082.04,-0.0001
2,2021-01-06,2,01,2968.21,2993.34,3027.16,2961.37,1.790000e+09,-0.0075,981.39,987.25,990.88,977.37,1.980000e+09,-0.0044,1087.93,1087.40,1089.79,1083.91,0.0014
3,2021-01-07,3,01,3031.68,2980.75,3055.28,2980.75,1.520000e+09,0.0214,988.86,983.28,993.91,982.27,2.260000e+09,0.0076,1094.28,1088.03,1096.78,1085.42,0.0058
4,2021-01-08,4,01,3152.18,3040.11,3161.11,3040.11,1.300000e+09,0.0397,987.79,990.70,995.22,978.12,2.560000e+09,-0.0011,1092.93,1094.35,1099.21,1088.79,-0.0012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,2021-12-08,2,49,3001.80,3017.93,3036.13,2995.34,4.882500e+08,0.0034,1006.04,1006.61,1012.64,1003.92,1.070000e+09,0.0094,1175.19,1176.85,1179.31,1175.12,-0.0013
243,2021-12-09,3,49,3029.57,3007.00,3029.57,3001.55,5.134400e+08,0.0093,1022.87,1009.29,1022.87,1009.29,1.060000e+09,0.0167,1178.15,1173.42,1179.37,1172.62,0.0025
244,2021-12-10,4,49,3010.23,3008.70,3017.64,2998.29,4.516000e+08,-0.0064,1011.57,1016.34,1018.42,1010.02,1.150000e+09,-0.0110,1180.86,1178.27,1182.82,1176.40,0.0023
245,2021-12-13,0,50,3001.66,3019.67,3043.83,3000.51,3.758300e+08,-0.0028,1005.96,1014.27,1014.90,1005.96,1.340000e+09,-0.0055,1184.91,1180.96,1186.16,1177.03,0.0034


In [16]:
#미국 국채 금리
DGS = fdr.DataReader(['DGS20','DGS10','DGS5','DGS1'], start_date, end_date, data_source='fred').reset_index()
DGS.rename(columns={'DATE': 'Date'}, inplace=True)
data = pd.merge(data, DGS, how='outer')
data

,Date,weekday,weeknum,kospi_Close,kospi_Open,kospi_High,kospi_Low,kospi_Volume,kospi_Change,kosdaq_Close,...,kosdaq_Change,exc_Close,exc_Open,exc_High,exc_Low,exc_Change,DGS20,DGS10,DGS5,DGS1
0,2021-01-04,0,01,2944.45,2874.50,2946.54,2869.11,1.030000e+09,0.0247,977.62,...,0.0095,1086.48,1085.73,1088.30,1080.02,0.0016,1.46,0.93,0.36,0.10
1,2021-01-05,1,01,2990.57,2943.67,2990.57,2921.84,1.520000e+09,0.0157,985.76,...,0.0083,1086.42,1086.69,1090.33,1082.04,-0.0001,1.49,0.96,0.38,0.10
2,2021-01-06,2,01,2968.21,2993.34,3027.16,2961.37,1.790000e+09,-0.0075,981.39,...,-0.0044,1087.93,1087.40,1089.79,1083.91,0.0014,1.60,1.04,0.43,0.11
3,2021-01-07,3,01,3031.68,2980.75,3055.28,2980.75,1.520000e+09,0.0214,988.86,...,0.0076,1094.28,1088.03,1096.78,1085.42,0.0058,1.64,1.08,0.46,0.11
4,2021-01-08,4,01,3152.18,3040.11,3161.11,3040.11,1.300000e+09,0.0397,987.79,...,-0.0011,1092.93,1094.35,1099.21,1088.79,-0.0012,1.67,1.13,0.49,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,2021-12-08,2,49,3001.80,3017.93,3036.13,2995.34,4.882500e+08,0.0034,1006.04,...,0.0094,1175.19,1176.85,1179.31,1175.12,-0.0013,1.93,1.52,1.27,0.29
243,2021-12-09,3,49,3029.57,3007.00,3029.57,3001.55,5.134400e+08,0.0093,1022.87,...,0.0167,1178.15,1173.42,1179.37,1172.62,0.0025,1.91,1.49,1.26,0.28
244,2021-12-10,4,49,3010.23,3008.70,3017.64,2998.29,4.516000e+08,-0.0064,1011.57,...,-0.0110,1180.86,1178.27,1182.82,1176.40,0.0023,1.91,1.48,1.25,0.27
245,2021-12-13,0,50,3001.66,3019.67,3043.83,3000.51,3.758300e+08,-0.0028,1005.96,...,-0.0055,1184.91,1180.96,1186.16,1177.03,0.0034,1.86,1.42,1.21,0.27


In [17]:
#나스닥(NASDAQ) : 4/10 20:00 빼기로 결정
#NASDAQ = fdr.DataReader('NASDAQCOM', start_date, end_date, data_source='fred').reset_index()
#NASDAQ.rename(columns={'DATE': 'Date'}, inplace=True)
#data = pd.merge(data, NASDAQ, how='outer')
#data

In [18]:
#S&P 500지수 => 4/10 20:00 빼기로 결정
#sp500 = fdr.DataReader('US500' ,start_date, end_date).reset_index()
#sp500.name='sp500'
#col_rename(sp500)

In [19]:
#data = pd.merge(data, sp500, how='outer')
#data

In [20]:
# 1년만기 한국국채 수익률
KR1YT_RR = fdr.DataReader('KR1YT=RR', start_date, end_date).reset_index()
KR1YT_RR.name='KR1YT_RR'
col_rename(KR1YT_RR)
KR1YT_RR

,Date,KR1YT_RR_Close,KR1YT_RR_Open,KR1YT_RR_High,KR1YT_RR_Low,KR1YT_RR_Change
0,2021-01-04,0.651,0.650,0.651,0.650,-0.0091
1,2021-01-05,0.636,0.635,0.636,0.635,-0.0230
2,2021-01-06,0.635,0.637,0.637,0.635,-0.0016
3,2021-01-07,0.636,0.635,0.636,0.635,0.0016
4,2021-01-08,0.633,0.635,0.635,0.632,-0.0047
...,...,...,...,...,...,...
333,2021-12-10,1.372,1.372,1.373,1.372,0.0059
334,2021-12-11,1.372,1.372,1.372,1.372,0.0000
335,2021-12-12,1.372,1.372,1.372,1.372,0.0000
336,2021-12-13,1.355,1.355,1.355,1.355,-0.0124


In [21]:
data = pd.merge(data, KR1YT_RR, how='outer')
data

,Date,weekday,weeknum,kospi_Close,kospi_Open,kospi_High,kospi_Low,kospi_Volume,kospi_Change,kosdaq_Close,...,exc_Change,DGS20,DGS10,DGS5,DGS1,KR1YT_RR_Close,KR1YT_RR_Open,KR1YT_RR_High,KR1YT_RR_Low,KR1YT_RR_Change
0,2021-01-04,0.0,01,2944.45,2874.50,2946.54,2869.11,1.030000e+09,0.0247,977.62,...,0.0016,1.46,0.93,0.36,0.10,0.651,0.650,0.651,0.650,-0.0091
1,2021-01-05,1.0,01,2990.57,2943.67,2990.57,2921.84,1.520000e+09,0.0157,985.76,...,-0.0001,1.49,0.96,0.38,0.10,0.636,0.635,0.636,0.635,-0.0230
2,2021-01-06,2.0,01,2968.21,2993.34,3027.16,2961.37,1.790000e+09,-0.0075,981.39,...,0.0014,1.60,1.04,0.43,0.11,0.635,0.637,0.637,0.635,-0.0016
3,2021-01-07,3.0,01,3031.68,2980.75,3055.28,2980.75,1.520000e+09,0.0214,988.86,...,0.0058,1.64,1.08,0.46,0.11,0.636,0.635,0.636,0.635,0.0016
4,2021-01-08,4.0,01,3152.18,3040.11,3161.11,3040.11,1.300000e+09,0.0397,987.79,...,-0.0012,1.67,1.13,0.49,0.10,0.633,0.635,0.635,0.632,-0.0047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,2021-11-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.230,1.230,1.230,1.230,0.0000
334,2021-12-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.406,1.406,1.406,1.406,0.0000
335,2021-12-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.406,1.406,1.406,1.406,0.0000
336,2021-12-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.372,1.372,1.372,1.372,0.0000


In [22]:
btx = fdr.DataReader('BTC/KRW',start_date,end_date).reset_index()
btx.name='btc_krw'
col_rename(btx)
btx

,Date,btc_krw_Close,btc_krw_Open,btc_krw_High,btc_krw_Low,btc_krw_Volume,btc_krw_Change
0,2021-01-04,36499000,37553000,38113000,36499000,2920.0,-0.0289
1,2021-01-05,37962000,36475000,37962000,35190000,4170.0,0.0401
2,2021-01-06,40731000,37989000,40731000,37989000,4250.0,0.0729
3,2021-01-07,44575000,40760000,44575000,40760000,5590.0,0.0944
4,2021-01-08,47241000,44592000,47241000,44592000,3880.0,0.0598
...,...,...,...,...,...,...,...
340,2021-12-10,59705000,59191000,61901000,58990000,730.0,0.0067
341,2021-12-11,61410000,59663000,61490000,59500000,490.0,0.0286
342,2021-12-12,61924000,61412000,62841000,60864000,480.0,0.0084
343,2021-12-13,58417000,61984000,62079000,58000000,750.0,-0.0566


In [23]:
data = pd.merge(data, btx, how='outer')
data

,Date,weekday,weeknum,kospi_Close,kospi_Open,kospi_High,kospi_Low,kospi_Volume,kospi_Change,kosdaq_Close,...,KR1YT_RR_Open,KR1YT_RR_High,KR1YT_RR_Low,KR1YT_RR_Change,btc_krw_Close,btc_krw_Open,btc_krw_High,btc_krw_Low,btc_krw_Volume,btc_krw_Change
0,2021-01-04,0.0,01,2944.45,2874.50,2946.54,2869.11,1.030000e+09,0.0247,977.62,...,0.650,0.651,0.650,-0.0091,36499000,37553000,38113000,36499000,2920.0,-0.0289
1,2021-01-05,1.0,01,2990.57,2943.67,2990.57,2921.84,1.520000e+09,0.0157,985.76,...,0.635,0.636,0.635,-0.0230,37962000,36475000,37962000,35190000,4170.0,0.0401
2,2021-01-06,2.0,01,2968.21,2993.34,3027.16,2961.37,1.790000e+09,-0.0075,981.39,...,0.637,0.637,0.635,-0.0016,40731000,37989000,40731000,37989000,4250.0,0.0729
3,2021-01-07,3.0,01,3031.68,2980.75,3055.28,2980.75,1.520000e+09,0.0214,988.86,...,0.635,0.636,0.635,0.0016,44575000,40760000,44575000,40760000,5590.0,0.0944
4,2021-01-08,4.0,01,3152.18,3040.11,3161.11,3040.11,1.300000e+09,0.0397,987.79,...,0.635,0.635,0.632,-0.0047,47241000,44592000,47241000,44592000,3880.0,0.0598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,2021-07-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,39569000,39150000,39814000,38976000,1870.0,0.0111
341,2021-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,46240000,47959000,49372000,46035000,940.0,-0.0368
342,2021-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,50757000,51015000,51733000,50163000,780.0,-0.0048
343,2021-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,54870000,55299000,55678000,53517000,680.0,-0.0078


In [24]:
# 금 ETF : 잠깐 비활성화
#ETF_KR = fdr.DataReader('GC',start_date,end_date).reset_index()
#ETF_KR.name='ETF_KR'
#col_rename(ETF_KR)
#data = pd.merge(data, ETF_KR, how='outer')
#data

In [25]:
data.columns

Index(['Date', 'weekday', 'weeknum', 'kospi_Close', 'kospi_Open', 'kospi_High',
       'kospi_Low', 'kospi_Volume', 'kospi_Change', 'kosdaq_Close',
       'kosdaq_Open', 'kosdaq_High', 'kosdaq_Low', 'kosdaq_Volume',
       'kosdaq_Change', 'exc_Close', 'exc_Open', 'exc_High', 'exc_Low',
       'exc_Change', 'DGS20', 'DGS10', 'DGS5', 'DGS1', 'KR1YT_RR_Close',
       'KR1YT_RR_Open', 'KR1YT_RR_High', 'KR1YT_RR_Low', 'KR1YT_RR_Change',
       'btc_krw_Close', 'btc_krw_Open', 'btc_krw_High', 'btc_krw_Low',
       'btc_krw_Volume', 'btc_krw_Change'],
      dtype='object')

In [26]:
data.isnull().sum()

Date                 0
weekday             98
weeknum             98
kospi_Close        109
kospi_Open         109
kospi_High         109
kospi_Low          109
kospi_Volume       109
kospi_Change       109
kosdaq_Close       109
kosdaq_Open        109
kosdaq_High        109
kosdaq_Low         109
kosdaq_Volume      109
kosdaq_Change      109
exc_Close           98
exc_Open            98
exc_High            98
exc_Low             98
exc_Change          98
DGS20               98
DGS10               98
DGS5                98
DGS1                98
KR1YT_RR_Close       7
KR1YT_RR_Open        7
KR1YT_RR_High        7
KR1YT_RR_Low         7
KR1YT_RR_Change      7
btc_krw_Close        0
btc_krw_Open         0
btc_krw_High         0
btc_krw_Low          0
btc_krw_Volume       0
btc_krw_Change       0
dtype: int64

In [27]:
#dji_nan = data[data['dji_Open'].isnull()].index
#data.columns

In [28]:
#for i in dji_nan:
#    for j in range(6,12):
#        n = data.iloc[i-1,-j]
#        data.iloc[i,-j] = n

In [29]:
# 한국 휴장 일은 data 에서 삭제.
kospi_nan = data[data['kospi_Open'].isnull()].index
exc_nan = data[data['exc_Open'].isnull()].index

In [30]:
nan_list = set(kospi_nan.to_list() + exc_nan.to_list())
data.drop(nan_list, inplace=True)
data.drop(0, axis=0, inplace=True)
shift_col = data.columns.drop(['Date', 'weekday', 'weeknum'])
data[shift_col] = data[shift_col].shift(5)
data = data[5:].reset_index(drop=True)

In [31]:
data.isnull().sum()

Date               0
weekday            0
weeknum            0
kospi_Close        0
kospi_Open         0
kospi_High         0
kospi_Low          0
kospi_Volume       0
kospi_Change       0
kosdaq_Close       0
kosdaq_Open        0
kosdaq_High        0
kosdaq_Low         0
kosdaq_Volume      0
kosdaq_Change      0
exc_Close          0
exc_Open           0
exc_High           0
exc_Low            0
exc_Change         0
DGS20              0
DGS10              0
DGS5               0
DGS1               0
KR1YT_RR_Close     0
KR1YT_RR_Open      0
KR1YT_RR_High      0
KR1YT_RR_Low       0
KR1YT_RR_Change    0
btc_krw_Close      0
btc_krw_Open       0
btc_krw_High       0
btc_krw_Low        0
btc_krw_Volume     0
btc_krw_Change     0
dtype: int64

In [32]:
data.isnull()

,Date,weekday,weeknum,kospi_Close,kospi_Open,kospi_High,kospi_Low,kospi_Volume,kospi_Change,kosdaq_Close,...,KR1YT_RR_Open,KR1YT_RR_High,KR1YT_RR_Low,KR1YT_RR_Change,btc_krw_Close,btc_krw_Open,btc_krw_High,btc_krw_Low,btc_krw_Volume,btc_krw_Change
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
226,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
227,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
228,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [33]:
#우리나라 장시작 전에, 미국이 장이 끝나기 때문에, 전날 미국장의 시작값과 당일 미국장의 끝값이 의미가 있을거라고 판단
# 시작값을 하나씩 뒤로 민다.

In [34]:
# 미국 휴장일은, 전날 데이터로 채우기
#dji_nan = data[data['dji_Open'].isnull()].index
#dji_nan

In [35]:
#for i in dji_nan:
#    for j in range(6,12):
#        n = data.iloc[i-1,-j]
#        data.iloc[i,-j] = n

In [36]:
#그래도 비어있음
data.isnull().sum()

Date               0
weekday            0
weeknum            0
kospi_Close        0
kospi_Open         0
kospi_High         0
kospi_Low          0
kospi_Volume       0
kospi_Change       0
kosdaq_Close       0
kosdaq_Open        0
kosdaq_High        0
kosdaq_Low         0
kosdaq_Volume      0
kosdaq_Change      0
exc_Close          0
exc_Open           0
exc_High           0
exc_Low            0
exc_Change         0
DGS20              0
DGS10              0
DGS5               0
DGS1               0
KR1YT_RR_Close     0
KR1YT_RR_Open      0
KR1YT_RR_High      0
KR1YT_RR_Low       0
KR1YT_RR_Change    0
btc_krw_Close      0
btc_krw_Open       0
btc_krw_High       0
btc_krw_Low        0
btc_krw_Volume     0
btc_krw_Change     0
dtype: int64

In [37]:
# 비어있는 부분을 앞/뒷날 평균으로 설정
#empty = ["dji_Close", "dji_Open", "dji_High", "dji_Low", "dji_Volume", "dji_Change"]
#for item in empty:
#    lost = data[data[item].isnull()].index.to_list()
#    for ind in lost:
        #(data.iloc[ind]).loc[item] = 0.5*(data.iloc[ind-1][item]+data.iloc[ind+1][item])
#        data.loc[ind,item] = data.loc[ind-1,item]+data.loc[ind+1,item]

In [38]:
#그래도 비어있음
#data.isnull().sum()
    

In [39]:
ms_data = data.copy()
data['weekday'] = data['weekday'].astype('str')
dummies = pd.get_dummies(data[['weekday', 'weeknum']], prefix=['weekday','weeknum'])
m_data = pd.concat([data, dummies], axis=1)
m_data.drop(['weekday','weeknum'], axis=1, inplace=True)
m_data

,Date,kospi_Close,kospi_Open,kospi_High,kospi_Low,kospi_Volume,kospi_Change,kosdaq_Close,kosdaq_Open,kosdaq_High,...,weeknum_41,weeknum_42,weeknum_43,weeknum_44,weeknum_45,weeknum_46,weeknum_47,weeknum_48,weeknum_49,weeknum_50
0,2021-01-12,2990.57,2943.67,2990.57,2921.84,1.520000e+09,0.0157,985.76,976.43,985.76,...,0,0,0,0,0,0,0,0,0,0
1,2021-01-13,2968.21,2993.34,3027.16,2961.37,1.790000e+09,-0.0075,981.39,987.25,990.88,...,0,0,0,0,0,0,0,0,0,0
2,2021-01-14,3031.68,2980.75,3055.28,2980.75,1.520000e+09,0.0214,988.86,983.28,993.91,...,0,0,0,0,0,0,0,0,0,0
3,2021-01-15,3152.18,3040.11,3161.11,3040.11,1.300000e+09,0.0397,987.79,990.70,995.22,...,0,0,0,0,0,0,0,0,0,0
4,2021-01-18,3148.45,3161.90,3266.23,3096.19,1.710000e+09,-0.0012,976.63,988.38,993.20,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,2021-12-08,2899.72,2860.12,2905.74,2837.03,5.639300e+08,0.0214,977.15,969.90,977.73,...,0,0,0,0,0,0,0,0,1,0
226,2021-12-09,2945.27,2874.64,2945.27,2874.64,5.344600e+08,0.0157,977.43,967.34,978.16,...,0,0,0,0,0,0,0,0,1,0
227,2021-12-10,2968.33,2935.93,2975.44,2927.55,4.867500e+08,0.0078,998.47,981.65,998.49,...,0,0,0,0,0,0,0,0,1,0
228,2021-12-13,2973.25,2954.82,2983.50,2932.49,4.799100e+08,0.0017,991.87,990.07,994.15,...,0,0,0,0,0,0,0,0,0,1


In [40]:
#옵션/선물만기일
f_option = ['2021-01-14', '2021-02-10', '2021-03-11', '2021-04-08', '2021-05-13', '2021-06-10', 
            '2021-07-08', '2021-08-12', '2021-09-09', '2021-10-14', '2021-11-11', '2021-12-09']
f_futures = ['2021-03-11', '2021-06-10', '2021-09-09', '2021-12-09']

In [41]:
# 모델링

In [42]:
sample_name = 'sample_submission.csv'
sample_submission = pd.read_csv(os.path.join(path, sample_name))
submission = sample_submission.set_index('Day')
xgb = XGBRegressor()
day_list = ['2021-11-29', '2021-11-30', '2021-12-01', '2021-12-02', '2021-12-03']
stock_list.drop(stock_list[stock_list['종목코드']=='031390'].index,inplace=True)

In [43]:
#warning 무시
import warnings
warnings.filterwarnings('ignore')

In [44]:
# dataframe 저장
m_data.to_csv("data.csv")

In [45]:
print(submission)

            000060  000080  000100  000120  000150  000240  000250  000270  \
Day                                                                          
2021-11-01       0       0       0       0       0       0       0       0   
2021-11-02       0       0       0       0       0       0       0       0   
2021-11-03       0       0       0       0       0       0       0       0   
2021-11-04       0       0       0       0       0       0       0       0   
2021-11-05       0       0       0       0       0       0       0       0   
2021-11-29       0       0       0       0       0       0       0       0   
2021-11-30       0       0       0       0       0       0       0       0   
2021-12-01       0       0       0       0       0       0       0       0   
2021-12-02       0       0       0       0       0       0       0       0   
2021-12-03       0       0       0       0       0       0       0       0   

            000660  000670  ...  330860  336260  336370  347860

In [46]:
##########################

In [47]:
stock_list['종목코드'].values

array(['005930', '000660', '035420', '035720', '207940', '005935',
       '051910', '006400', '005380', '068270', '000270', '005490',
       '066570', '028260', '012330', '096770', '051900', '017670',
       '105560', '055550', '034730', '036570', '011200', '015760',
       '032830', '018260', '009150', '090430', '086790', '251270',
       '003670', '352820', '034020', '010950', '033780', '003490',
       '000810', '010130', '009540', '326030', '011170', '030200',
       '018880', '034220', '316140', '009830', '024110', '086280',
       '004020', '097950', '032640', '011780', '021240', '000720',
       '161390', '006800', '035250', '071050', '267250', '011070',
       '008930', '002790', '028050', '139480', '271560', '241560',
       '180640', '000100', '069500', '000120', '078930', '003410',
       '029780', '028670', '004990', '005830', '016360', '128940',
       '005387', '298020', '006360', '020150', '007070', '008770',
       '005940', '112610', '042660', '307950', '006280', '0085

In [48]:
code = '005930'

In [49]:
day_list

['2021-11-29', '2021-11-30', '2021-12-01', '2021-12-02', '2021-12-03']

In [50]:
day = '2021-11-29'

In [51]:
code_data = fdr.DataReader(code, start = start_date, end = end_date).reset_index()
code_data['y_Close'] = code_data['Close']  

In [52]:
code_data

,Date,Open,High,Low,Close,Volume,Change,y_Close
0,2021-01-04,81000,84400,80200,83000,38655276,0.024691,83000
1,2021-01-05,81600,83900,81600,83900,35335669,0.010843,83900
2,2021-01-06,83300,84500,82100,82200,42089013,-0.020262,82200
3,2021-01-07,82800,84200,82700,82900,32644642,0.008516,82900
4,2021-01-08,83300,90000,83000,88800,59013307,0.071170,88800
...,...,...,...,...,...,...,...,...
231,2021-12-08,78300,78600,77100,77400,21558340,0.000000,77400
232,2021-12-09,77400,78200,77000,78200,21604528,0.010336,78200
233,2021-12-10,77400,77600,76800,76900,9155219,-0.016624,76900
234,2021-12-13,77200,78300,76500,76800,15038750,-0.001300,76800


In [53]:
for code in tqdm(stock_list['종목코드'].values):
    for day in day_list:
        #print(f"day:{day}")
        code_data = fdr.DataReader(code, start = start_date, end = end_date).reset_index()
        code_data['y_Close'] = code_data['Close']    
        
        #code_data[[
        #    'Open', 'High', 'Low', 'Close', 'Volume', 'Change']] = code_data[[
        #    'Open', 'High', 'Low', 'Close', 'Volume', 'Change']].shift(5)
        
        #이동 평균선 추가: 60일
        code_data['60_close'] = code_data['Close'].rolling(window=60).mean() 

        #dacon 자료
        
        #이동 평균선 추가 (5/10/20)
        code_data['5_close'] = code_data['Close'].rolling(window=5).mean()   
        code_data['10_close'] = code_data['Close'].rolling(window=10).mean()   
        code_data['20_close'] = code_data['Close'].rolling(window=20).mean()   
        
        #6일전/7일전 종가 추가
        code_data['6d_Close'] = code_data[['Close']].shift(1)
        code_data['7d_Close'] = code_data[['6d_Close']].shift(1)
        
        #6일전/7일전 종가와 전일 종가의 차이 비율 추가
        code_data['diff_Close'] = (code_data['Close'] - code_data['6d_Close'])/code_data['Close']
        code_data['diff_Close2'] = (code_data['Close'] - code_data['7d_Close'])/code_data['Close']

        
        #6일전 거래량 추가
        code_data['6d_Volume'] = code_data[['Volume']].shift(6)
        
        #2일전 거래량과 전일 거래량의 차이 추가
        code_data['diff_Volume'] = code_data['Volume'] - code_data['6d_Volume']
               
        df = pd.merge(m_data, code_data, how='left', on='Date')
        df = df.drop(df[df.isnull().any(axis=1)].index, axis=0)
       
        
       
        market = stock_list[stock_list['종목코드'].str.contains(code)]['상장시장'].values[0]
        
        if market == 'KOSPI':
            df_kospi = df.drop(['kosdaq_Close', 'kosdaq_Open', 'kosdaq_High', 'kosdaq_Low', 'kosdaq_Volume', 'kosdaq_Change'], axis=1)
            #print(df_kospi[df_kospi['Date'] < day].drop(['Date','y_Close'], axis=1))
            #print(df_kospi[df_kospi['Date'] < day]['y_Close'])
            x_train = df_kospi[df_kospi['Date'] < day].drop(['Date','y_Close'], axis=1)
            y_train = df_kospi[df_kospi['Date'] < day]['y_Close']
            xgb.fit(x_train, y_train)
            #(df_kospi[df_kospi['Date']==day].drop(['Date','y_Close'], axis=1)).to_csv(f"data_{day}.csv")
            pred = xgb.predict(df_kospi[df_kospi['Date']==day].drop(['Date','y_Close'], axis=1))
        else:
            df_kosdaq = df.drop(['kospi_Close', 'kospi_Open', 'kospi_High', 'kospi_Low','kospi_Volume', 'kospi_Change', 'kosdaq_Close', 'kosdaq_Open'], axis=1)
            x_train = df_kosdaq[df_kosdaq['Date'] < day].drop(['Date','y_Close'], axis=1)
            y_train = df_kosdaq[df_kosdaq['Date'] < day]['y_Close']
            xgb.fit(x_train, y_train)
            pred = xgb.predict(df_kosdaq[df_kosdaq['Date']==day].drop(['Date','y_Close'], axis=1))
        #if len(pred):
        #print(pred)
        #print("============================================================================")
        submission.loc[day,code] = pred

100%|████████████████████████████████████████████████████████████████████████████████| 369/369 [05:15<00:00,  1.17it/s]


In [54]:
submission

,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
Day,,,,,,,,,,,,,,,,,,,,,
2021-11-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2021-11-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2021-11-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2021-11-04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2021-11-05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2021-11-29,31170.013672,30409.515625,56112.476562,129736.476562,120753.250000,15547.195312,43804.003906,77226.437500,114788.015625,640816.8125,...,43262.769531,48955.578125,95712.195312,37133.710938,21122.498047,49589.199219,372159.96875,262501.843750,24942.351562,17410.691406
2021-11-30,32246.482422,30291.195312,55420.476562,129317.289062,110611.078125,15550.885742,43362.820312,77787.726562,115860.765625,641118.8750,...,43564.390625,49124.511719,96666.312500,34816.585938,19738.228516,49130.726562,363165.31250,258045.390625,24224.587891,19466.125000
2021-12-01,31111.978516,30289.941406,54992.640625,126366.218750,110869.953125,15199.698242,42841.585938,80933.382812,115397.171875,633423.9375,...,48746.902344,48602.351562,94940.234375,36395.675781,19168.894531,47934.125000,364652.87500,263724.125000,24011.525391,18661.152344
2021-12-02,31875.173828,30475.988281,57385.394531,125409.195312,111752.476562,16069.200195,42899.109375,81164.593750,120338.343750,635044.6875,...,52402.046875,46703.011719,92741.382812,34677.488281,20375.496094,46783.144531,336394.59375,265715.906250,24298.250000,18649.054688


In [55]:
submission = submission.iloc[5:,:]
submission

,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
Day,,,,,,,,,,,,,,,,,,,,,
2021-11-29,31170.013672,30409.515625,56112.476562,129736.476562,120753.250000,15547.195312,43804.003906,77226.437500,114788.015625,640816.8125,...,43262.769531,48955.578125,95712.195312,37133.710938,21122.498047,49589.199219,372159.96875,262501.843750,24942.351562,17410.691406
2021-11-30,32246.482422,30291.195312,55420.476562,129317.289062,110611.078125,15550.885742,43362.820312,77787.726562,115860.765625,641118.8750,...,43564.390625,49124.511719,96666.312500,34816.585938,19738.228516,49130.726562,363165.31250,258045.390625,24224.587891,19466.125000
2021-12-01,31111.978516,30289.941406,54992.640625,126366.218750,110869.953125,15199.698242,42841.585938,80933.382812,115397.171875,633423.9375,...,48746.902344,48602.351562,94940.234375,36395.675781,19168.894531,47934.125000,364652.87500,263724.125000,24011.525391,18661.152344
2021-12-02,31875.173828,30475.988281,57385.394531,125409.195312,111752.476562,16069.200195,42899.109375,81164.593750,120338.343750,635044.6875,...,52402.046875,46703.011719,92741.382812,34677.488281,20375.496094,46783.144531,336394.59375,265715.906250,24298.250000,18649.054688
2021-12-03,32087.873047,29520.546875,57453.707031,130240.140625,110035.609375,16577.583984,44644.687500,82403.359375,116832.062500,645937.6875,...,51449.867188,46776.960938,91999.179688,34448.449219,18965.675781,48728.332031,346507.75000,273345.187500,25603.044922,18177.464844


In [56]:
true_close = sample_submission.copy().set_index('Day')

code_data = fdr.DataReader('000060', start = '2021-11-01', end = '2021-11-01')['Close'].reset_index()

In [57]:
for code in tqdm(stock_list['종목코드'].values):
    for day in day_list:
        code_data = fdr.DataReader(code, start = day, end = day)['Close'].reset_index()
        true_close.loc[day,code] = code_data['Close'].values[0]

100%|████████████████████████████████████████████████████████████████████████████████| 369/369 [02:02<00:00,  3.00it/s]


In [58]:
true_close = true_close.iloc[5:,:]

In [59]:
true_close

,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
Day,,,,,,,,,,,,,,,,,,,,,
2021-11-29,31200,30300,56369,129000,110500,15200,43250,79200,116000,641000,...,43750,48750,96859,37100,19800,49100,369000,266300,24200,17650
2021-11-30,31300,29000,55317,124000,108500,15150,42150,77800,114000,632000,...,43300,49250,94257,35550,18600,45500,364500,255800,23100,19500
2021-12-01,31700,29400,55221,125000,112000,15500,42750,81200,116500,630000,...,48950,48700,94931,36050,18900,46200,352500,264200,23850,18650
2021-12-02,32150,29550,57518,129000,110000,16150,43100,81600,120000,639000,...,51900,46250,91654,33800,18650,48550,330000,274700,25200,18050
2021-12-03,32700,30600,57709,131000,108500,16400,44900,82500,118000,642000,...,51900,46800,91847,34500,19100,49000,354500,275900,25800,18150


In [60]:
diff = np.mean(np.abs(submission-true_close)/true_close)

In [61]:
np.mean(np.mean(np.abs(submission[:5] - true_close[:5])/true_close[:5]))*100

1.6380163886255024